# Sandstone 2D parallel-beam data reconstruction demo



In [ ]:
# Import all CIL components needed
from cil.framework import ImageData, ImageGeometry
from cil.framework import AcquisitionGeometry, AcquisitionData

# CIL Processors
from cil.processors import CentreOfRotationCorrector, Slicer, TransmissionAbsorptionConverter, Normaliser #, Padder

from cil.utilities.display import show2D

# Import from cil.plugins.astra
from cil.plugins.astra.processors import FBP
from cil.plugins.astra.operators import ProjectionOperator

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

Load demo data set and display the first raw projection

In [ ]:
all_data = scipy.io.loadmat("/media/newhd/shared/Data/synchrotron/small/slice_0270_data.mat")
#all_data = scipy.io.loadmat("/media/newhd/shared/Data/synchrotron/small/slice_0540_data.mat")
#all_data = scipy.io.loadmat("/media/newhd/shared/Data/synchrotron/small/slice_0810_data.mat")
#all_data = scipy.io.loadmat("/media/newhd/shared/Data/synchrotron/small/slice_1080_data.mat")

In [ ]:
darks = all_data['X_dark']
flats = all_data['X_flat']
projs = all_data['X_proj']
projs.shape

In [ ]:
plt.imshow(projs)

In [ ]:
denom = (flats.mean(axis=1) - darks.mean(axis=1))
denom.shape = (2560,1)
denom.shape

In [ ]:
projs2 = projs/denom

In [ ]:
projs.shape

Manually pad for centre-of-rotation correction

In [ ]:
projs2pad = np.pad(projs2,((0,88),(0,0)),'edge')

In [ ]:
plt.imshow(projs2pad)
projs2pad.shape

Also pad on both sides for region-of-interest correction

In [ ]:
padsize = 500
projs3pad = np.pad(projs2pad,((padsize,padsize),(0,0)),'edge')
projs3pad.shape

In [ ]:
plt.imshow(projs3pad)

In [ ]:
ag = AcquisitionGeometry.create_Parallel2D()  \
         .set_panel(num_pixels=(2560+88+2*padsize))        \
         .set_angles(angles=np.linspace(0,180, 1500))

In [ ]:
data = AcquisitionData(np.transpose(projs3pad),False,geometry=ag)

In [ ]:
#data2 = Normaliser(flat_field=flats,dark_field=darks)(data)

In [ ]:
data2 = TransmissionAbsorptionConverter()(data)

In [ ]:
CentreOfRotationCorrector.xcorr()(data2)

In [ ]:
ig = ag.get_ImageGeometry()

In [ ]:
ig.voxel_num_x = 2560
ig.voxel_num_y = 2560
print(ig)

In [ ]:
fbp = FBP(ig,ag,device='gpu')

In [ ]:
rec1 = fbp(data2)

In [ ]:
show2D(rec1)

Load and plot the reconstruction provided from the synchrotron (rotate to match our orientation)

In [ ]:
vendor_recon = plt.imread("/media/newhd/shared/Data/synchrotron/small/recon/BBii_0270.rec.16bit.tif")
#vendor_recon = plt.imread("/media/newhd/shared/Data/synchrotron/small/recon/BBii_0540.rec.16bit.tif")
#vendor_recon = plt.imread("/media/newhd/shared/Data/synchrotron/small/recon/BBii_0810.rec.16bit.tif")
#vendor_recon = plt.imread("/media/newhd/shared/Data/synchrotron/small/recon/BBii_1080.rec.16bit.tif")

In [ ]:
show2D(np.rot90(vendor_recon))